# Análisis de Rentabilidad y Riesgo para Nuevos Pozos Petrolíferos — *OilyGiant*

---

## **Objetivo del Proyecto**

El objetivo de este proyecto es **identificar la mejor región para abrir 200 nuevos pozos de petróleo** en la compañía **OilyGiant**.
Para ello, se utilizarán modelos de *machine learning* (regresión lineal) que predigan el volumen de reservas en cada pozo potencial, combinados con análisis financiero y estadístico mediante **bootstrapping**.

El proyecto busca responder a tres preguntas fundamentales:

1. ¿Qué región tiene el **mayor beneficio esperado** si se desarrollan los 200 pozos más prometedores?
2. ¿Cuál es el **riesgo de pérdida** (probabilidad de ganancia negativa) en cada región?
3. ¿Qué región cumple simultáneamente con **mayor rentabilidad y menor riesgo (< 2.5%)**?


##  Flujo de Trabajo General

1. Comprender el negocio y las restricciones.  
2. Preparar y explorar los datos.  
3. Entrenar y evaluar el modelo (regresión lineal).  
4. Calcular beneficio potencial.  
5. Evaluar riesgo con bootstrapping.  
6. Seleccionar la región óptima.  
7. Redactar conclusiones finales.

*Nota: Se realizará el proceso completo para una región, después se implementarán funciones para optimizar el flujo y aplicarlo a las dos regiones restantes.*


# **Comprensión del Negocio y las Restricciones**

* Analizar las condiciones del problema:

  * Tres regiones (archivos independientes).
  * 500 pozos por región; se perforarán los **200 mejores**.
  * Presupuesto total: **100 millones USD**.
  * Precio por mil barriles: **4.5 mil USD**.
  * Límite de riesgo aceptable: **2.5% de probabilidad de pérdida**.
* Identificar las variables clave:

  * `f0`, `f1`, `f2` → características geológicas.
  * `product` → volumen de reservas (en miles de barriles).

---

# **Preparación y Exploración de Datos**
* Importar librerías necesarias (`pandas`, `numpy`, `sklearn`).
* Cargar los tres datasets:
  `geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv`.
* Revisar estructura y calidad de los datos (tipos, nulos, duplicados).
* Analizar estadísticas básicas y correlaciones.
* Separar características (`f0`, `f1`, `f2`) y objetivo (`product`).
* Dividir cada dataset en **entrenamiento (75%)** y **validación (25%)**.



In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#Random seed for reproducibility
RANDOM_SEED = 12345

In [2]:
# Cargar los datasets
data_0 = pd.read_csv('../data/geo_data_0.csv')
data_1 = pd.read_csv('../data/geo_data_1.csv')
data_2 = pd.read_csv('../data/geo_data_2.csv')

In [3]:
# Revisar estructura y calidad de los datos
for i, data in enumerate([data_0, data_1, data_2]):
    print(f"Dataset {i}:")
    display(data.info())
    display(data.head())
    display(data.describe())
    print(f"Nulos:\n{data.isnull().sum()}")
    print(f"Duplicados: {data.duplicated().sum()}\n")

Dataset 0:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


Nulos:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Duplicados: 0

Dataset 1:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

Nulos:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Duplicados: 0

Dataset 2:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Nulos:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Duplicados: 0



In [4]:
# Separar características y objetivo y dividir en entrenamiento y validación 75/25
X_0 = data_0[['f0', 'f1', 'f2']]
y_0 = data_0['product']
X_train_0, X_val_0, y_train_0, y_val_0 = train_test_split(X_0, y_0, test_size=0.25, random_state=RANDOM_SEED)




# **Entrenamiento del Modelo (Regresión Lineal)**

* Entrenar un modelo de **regresión lineal** para cada región.
* Obtener predicciones sobre el conjunto de validación.
* Calcular:

  * Volumen medio real (`product.mean()`).
  * Volumen medio predicho.
  * Error RMSE (raíz del error cuadrático medio).
* Guardar los resultados de cada región:

  * Valores reales (`target_valid`).
  * Predicciones (`pred_valid`).

>  *En esta fase se creará la primera función modular: entrenamiento y evaluación del modelo por región.*


In [5]:
# Primero seguimos el flujo completo para el primer dataset, después crearé la función para automatizar el proceso
# Crear un pipeline con preprocesamiento y modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
# Entrenar el modelo
pipeline.fit(X_train_0, y_train_0)
# Hacer predicciones
pred_val_0 = pipeline.predict(X_val_0)
# Evaluar el modelo
rmse_0 = root_mean_squared_error(y_val_0, pred_val_0)
# Volumen medio de reservas predicho y valor rmse
pred_vol_mean_0 = pred_val_0.mean()
print(f"Volumen medio de reservas predicho para el dataset 0: {pred_vol_mean_0:.2f}")
print(f"RMSE para el dataset 0: {rmse_0:.2f}")
# Volumen medio real de reservas
real_vol_mean_0 = y_val_0.mean()
print(f"Volumen medio real de reservas para el dataset 0: {real_vol_mean_0:.2f}")

Volumen medio de reservas predicho para el dataset 0: 92.59
RMSE para el dataset 0: 37.58
Volumen medio real de reservas para el dataset 0: 92.08


In [6]:
# Crear la función para automatizar el proceso
def train_and_evaluate(data:pd.DataFrame, region_name: str, verbose=False):
    '''Entrena una regresión lineal con estandarización para una región,
    evalúa en el conjunto de validación y devuelve predicciones y métricas.

    Parámetros
    ----------
    data: DataFrame con columnas ['id', 'f0','f1','f2','product'].
    region_name : etiqueta de la región para trazabilidad.
    verbose : si True, imprime métricas y promedios.

    Retorna
    -------
    dict con:
      - 'region': nombre de la región
      - 'rmse': RMSE en validación (float)
      - 'mean_true': media real del objetivo en validación (float)
      - 'mean_pred': media predicha en validación (float)
      - 'valid_df': DataFrame con columnas ['y_true','y_pred'] (índices alineados)
     '''
    X = data[['f0','f1','f2']]
    y= data['product']
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)
    pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('model', LinearRegression())
        ])
    pipe.fit(X_train,y_train)
    y_pred = pipe.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mean_true = y_val.mean()
    mean_pred = y_pred.mean()
    valid_df = pd.DataFrame({'y_val': y_val,
                             'y_pred': y_pred}, index=y_val.index)
    if verbose:
        print(f'Región : {region_name}')
        print(f'RMSE del modelo: {rmse:.2f}')
        print(f'Volumen real promedio: {mean_true:.2f}')
        print(f'Volumen predicho promedio: {mean_pred:.2f}')
    return {'region': region_name,
            'rmse': rmse,
            'mean_true': mean_true,
            'mean_pred': mean_pred,
            'valid_df': valid_df}

In [7]:
# Comprobamos que la función funciona correctamente comparando con los valores que hicimos manualmente
results_0 = train_and_evaluate(data_0, 'Región 0', verbose=True)

Región : Región 0
RMSE del modelo: 37.58
Volumen real promedio: 92.08
Volumen predicho promedio: 92.59


In [8]:
# Resultados para la región 1
results_1 = train_and_evaluate(data_1, 'Región 1', verbose=True)

Región : Región 1
RMSE del modelo: 0.89
Volumen real promedio: 68.72
Volumen predicho promedio: 68.73


In [9]:
# Resultados para la región 2
results_2 = train_and_evaluate(data_2, 'Región 2', verbose=True)

Región : Región 2
RMSE del modelo: 40.03
Volumen real promedio: 94.88
Volumen predicho promedio: 94.97



##  Conclusión

| Región | RMSE     | Volumen medio | Interpretación                                  |
| ------ | -------- | ------------- | ----------------------------------------------- |
| 0      | 37.58    | 92.08         | Buen equilibrio, pero con error moderado.       |
| 1      | **0.89** | **68.72**     | Extremadamente precisa pero de baja producción. |
| 2      | 40.03    | 94.88         | Alta producción, error algo mayor.              |

 **Hipótesis actual:**
La **Región 0 o 2** serán las candidatas principales para perforación,
pero se confirmará tras el cálculo de **beneficio y riesgo (bootstrapping)**.

---


# **Cálculo del Beneficio Potencial**

* Recordar que:

  * 200 pozos × 0.5 millones USD = **100 millones USD de inversión total.**
  * 1 unidad (`product = 1`) equivale a **4500 USD**.
  * Punto de equilibrio: **111.1 unidades promedio por pozo.**
* Seleccionar los **200 pozos con mayor volumen predicho**.
* Calcular el **beneficio total esperado** por región:
  $$
  \text{beneficio} = (\text{suma de reservas reales} \times 4500) - 100,000,000
  $$
* Comparar las tres regiones según su beneficio inicial.

>  *En esta fase se crea una función para calcular el beneficio económico de cada región.*

In [10]:
# Parámetros económicos
BUDGET = 100_000_000             # USD (inversión total)
REVENUE_PER_UNIT = 4_500         # USD por unidad de 'product' (mil barriles)
TOP_200 = 200                      # pozos a perforar
BREAK_EVEN_UNITS = (BUDGET/TOP_200) / REVENUE_PER_UNIT  # unidades necesarias para cubrir costes
SAMPLE_SIZE = 500                # tamaño de muestra de exploración (para bootstrap)
# Calcular el beneficio potencial de los 200 mejores pozos de cada región mediante una función
def calc_potential_profit(valid_df: pd.DataFrame, region_name: str, verbose=False):
    '''Calcula el beneficio potencial de perforar los 200 mejores pozos, además de otras métricas de negocio,
    según las predicciones del modelo.
    Parámetros
    ----------
    valid_df : pd.DataFrame
        DataFrame que contiene las predicciones y valores reales de los pozos.
    region_name : str
        Nombre de la región para la que se calcula el beneficio potencial.
    verbose : bool, opcional
        Si es True, imprime información detallada sobre el cálculo.
     Retorna
    -------
    dict con:
      - 'region': nombre de la región
      - 'total_true': suma real de 'product' de los 200 mejores pozos (float)
      - 'gross_revenue': ingreso bruto estimado (float)
      - 'profit': beneficio potencial (float)
      - 'avg_true': producción promedio por pozo (float)  
    '''
    # Ordenar por predicción y tomar los mejores 200
    best_200 = valid_df.sort_values(by='y_pred', ascending=False).head(200)['y_val']
    # Calcular beneficio y el promedio de producción por pozo para compararlo con el punto de equilibrio
    total_true = best_200.sum()
    avg_true = best_200.mean()
    gross_revenue = total_true * REVENUE_PER_UNIT
    profit = gross_revenue - BUDGET
    if verbose:
        print(f'Región: {region_name}')
        print(f"Suma real top-{TOP_200} (unidades): {total_true:,.2f}")
        print(f"Ingreso bruto estimado (USD): ${gross_revenue:,.2f}")
        print(f'Beneficio potencial al perforar los 200 mejores pozos: ${profit:,.2f}')
        print(f'Producción promedio por pozo: {avg_true:.2f} unidades')
        if avg_true >= BREAK_EVEN_UNITS:
            print(f'Producción promedio por pozo está por encima del punto de equilibrio ({BREAK_EVEN_UNITS:.2f} unidades).')
        else:
            print(f'Producción promedio por pozo está por debajo del punto de equilibrio ({BREAK_EVEN_UNITS:.2f} unidades).')
    
  
    return {'region': region_name,
        'total_true': total_true,
        'gross_revenue': gross_revenue,
        'profit': profit,
        'avg_true': avg_true}


In [11]:
# Calculamos las métricas para la región 0
profit_0 = calc_potential_profit(results_0['valid_df'], 'Región 0', verbose=True)

Región: Región 0
Suma real top-200 (unidades): 29,601.84
Ingreso bruto estimado (USD): $133,208,260.43
Beneficio potencial al perforar los 200 mejores pozos: $33,208,260.43
Producción promedio por pozo: 148.01 unidades
Producción promedio por pozo está por encima del punto de equilibrio (111.11 unidades).


In [12]:
# Calculamos las métricas para la región 1
profit_1 = calc_potential_profit(results_1['valid_df'], 'Región 1', verbose=True)

Región: Región 1
Suma real top-200 (unidades): 27,589.08
Ingreso bruto estimado (USD): $124,150,866.97
Beneficio potencial al perforar los 200 mejores pozos: $24,150,866.97
Producción promedio por pozo: 137.95 unidades
Producción promedio por pozo está por encima del punto de equilibrio (111.11 unidades).


In [13]:
# Calculamos las métricas para la región 2
profit_2 = calc_potential_profit(results_2['valid_df'], 'Región 2', verbose=True)

Región: Región 2
Suma real top-200 (unidades): 28,245.22
Ingreso bruto estimado (USD): $127,103,499.64
Beneficio potencial al perforar los 200 mejores pozos: $27,103,499.64
Producción promedio por pozo: 141.23 unidades
Producción promedio por pozo está por encima del punto de equilibrio (111.11 unidades).


## Observaciones
* La **Región 0** es la que muestra el mayor beneficio esperado, seguida de la **Región 2**.
* La **Región 1**, a pesar de su alta precisión, es la que genera menos unidades y, por ende, el menor beneficio.
* Todas las regiones superan el punto de equilibrio, pero la **Región 0** lo hace con mayor margen.
- Hipótesis actual: La **Región 0** es la mejor candidata, pero se confirmará tras el análisis de riesgo.

# **Evaluación de Riesgo con Bootstrapping**

* Aplicar **bootstrapping (1000 iteraciones)** usando los datos de validación de cada región.
* En cada iteración:

  * Crear una muestra aleatoria con reemplazo.
  * Seleccionar los 200 pozos con mayor valor predicho.
  * Calcular el beneficio total.
* A partir de las 1000 simulaciones, estimar:

  * **Beneficio promedio (esperado).**
  * **Intervalo de confianza (95%).**
  * **Probabilidad de pérdida (beneficio < 0).**
* Guardar resultados en una tabla resumen por región.

>  *Aquí se desarrolla la función de bootstrapping que automatiza el cálculo de distribución de beneficios y riesgo.*


In [14]:
# Creamos la función para análisis de riesgo mediante bootstrap
def risk_analysis(valid_df: pd.DataFrame, region_name: str, n_iterations=1000, sample_size=SAMPLE_SIZE, verbose=False):
    
    rng = np.random.default_rng(RANDOM_SEED)
    n = len(valid_df)
    profit_samples = np.empty(n_iterations)
    for i in range(n_iterations):
        sample_indices = rng.choice(n, size=sample_size, replace=True)
        sample_df = valid_df.iloc[sample_indices]
        profit_info = calc_potential_profit(sample_df, region_name)
        profit_samples[i] = profit_info['profit']
        
    mean_profit = profit_samples.mean()
    prob_loss = np.mean(profit_samples < 0)*100
    ci_low, ci_high = np.percentile(profit_samples, [2.5, 97.5])
    if verbose:
        print(f'Análisis de riesgo para {region_name}:')
        print(f'Beneficio promedio estimado: ${mean_profit:,.2f}')
        print(f'Probabilidad de pérdida: {prob_loss:.2f}%')
        print(f'Intervalo de confianza del 95% para el beneficio: (${ci_low:,.2f}, ${ci_high:,.2f})')
    return {'region': region_name,
            'mean_profit': mean_profit,
            'prob_loss': prob_loss,
            'ci_low': ci_low,
            'ci_high': ci_high,
            'profit_samples': profit_samples}
    

In [15]:
# Realizamos el análisis de riesgo para la región 0
risk_analysis_0 = risk_analysis(results_0['valid_df'], 'Región 0', verbose=True)

Análisis de riesgo para Región 0:
Beneficio promedio estimado: $3,690,526.73
Probabilidad de pérdida: 8.50%
Intervalo de confianza del 95% para el beneficio: ($-1,767,205.24, $8,869,696.17)


In [16]:
# Realizamos el análisis de riesgo para la región 1
risk_analysis_1 = risk_analysis(results_1['valid_df'], 'Región 1', verbose=True)

Análisis de riesgo para Región 1:
Beneficio promedio estimado: $4,541,582.12
Probabilidad de pérdida: 1.40%
Intervalo de confianza del 95% para el beneficio: ($488,526.54, $8,459,901.66)


In [17]:
# Realizamos el análisis de riesgo para la región 2
risk_analysis_2 = risk_analysis(results_2['valid_df'], 'Región 2', verbose=True)

Análisis de riesgo para Región 2:
Beneficio promedio estimado: $3,788,328.73
Probabilidad de pérdida: 6.00%
Intervalo de confianza del 95% para el beneficio: ($-1,350,655.68, $8,823,111.12)


## Observaciones
- La **región 0** presenta el menor beneficio promedio con un riesgo de pérdida del 8.5%, es bastante más de lo aceptable (2.5%). Además, su intervalo de confianza incluye valores negativos, lo que indica una alta incertidumbre.
- La **región 1** tiene el mayor beneficio promedio con el menor riesgo de pérdida del 1.4%, cumpliendo con el criterio de riesgo. Su intervalo de confianza es positivo, mostrando estabilidad.
- La **región 2** tiene un beneficio promedio intermedio pero con un riesgo de pérdida del 6%, que excede el límite aceptable. Su intervalo de confianza también incluye valores negativos.

- Hipótesis final: La **Región 1** es la mejor opción, ya que combina alta rentabilidad con bajo riesgo, cumpliendo los criterios establecidos. Contrasta con la hipótesis inicial basada solo en beneficio esperado. Resaltando la importancia de considerar tanto rentabilidad como riesgo en decisiones de inversión.

# **Selección de la Región Óptima**

* Crear una tabla comparativa con los resultados de cada región:

  * RMSE
  * Beneficio medio
  * Intervalo de confianza (95%)
  * Riesgo de pérdida (%)
* Mantener solo regiones con **riesgo < 2.5%**.
* Elegir la región con **mayor beneficio promedio**.
* Verificar si coincide con la elección inicial del análisis sin bootstrap.


In [18]:

# Reunimos los resultados en una lista para iterar
regions_data = [
    ("Región 0", results_0, profit_0, risk_analysis_0),
    ("Región 1", results_1, profit_1, risk_analysis_1),
    ("Región 2", results_2, profit_2, risk_analysis_2),
]

# Creamos el resumen 
summary_df = pd.DataFrame([
    {
        "Región": name,
        "RMSE": res["rmse"],
        "Volumen Real Promedio": res["mean_true"],
        "Volumen Predicho Promedio": res["mean_pred"],
        "Beneficio Potencial (M USD)": prof["profit"] / 1_000_000,
        "Producción Promedio (unidades)": prof["avg_true"],
        "Beneficio Promedio Estimado (M USD)": risk["mean_profit"] / 1_000_000,
        "Probabilidad de Pérdida (%)": risk["prob_loss"],
        "IC 95% Beneficio (M USD)": f"({risk['ci_low']/1_000_000:,.2f}, {risk['ci_high']/1_000_000:,.2f})"
    }
    for name, res, prof, risk in regions_data
])

summary_df


,Región,RMSE,Volumen Real Promedio,Volumen Predicho Promedio,Beneficio Potencial (M USD),Producción Promedio (unidades),Beneficio Promedio Estimado (M USD),Probabilidad de Pérdida (%),IC 95% Beneficio (M USD)
0,Región 0,37.579422,92.078597,92.592568,33.208260,148.009178,3.690527,8.5,"(-1.77, 8.87)"
1,Región 1,0.893099,68.723136,68.728547,24.150867,137.945408,4.541582,1.4,"(0.49, 8.46)"
2,Región 2,40.029709,94.884233,94.965046,27.103500,141.226111,3.788329,6.0,"(-1.35, 8.82)"


In [19]:
# Mantenemos regiones con <=2.5% de riesgo de pérdida
final_candidates = summary_df[summary_df["Probabilidad de Pérdida (%)"] <= 2.5]
final_candidates

,Región,RMSE,Volumen Real Promedio,Volumen Predicho Promedio,Beneficio Potencial (M USD),Producción Promedio (unidades),Beneficio Promedio Estimado (M USD),Probabilidad de Pérdida (%),IC 95% Beneficio (M USD)
1,Región 1,0.893099,68.723136,68.728547,24.150867,137.945408,4.541582,1.4,"(0.49, 8.46)"


# **Conclusión final - Selección de Región para Nuevos Pozos Petrolíferos**


Tras el análisis de los modelos predictivos y la evaluación del riesgo mediante *bootstrapping*, la **Región 1** fue seleccionada como la mejor candidata para el desarrollo de los **200 nuevos pozos petrolíferos**.

---

## **Región seleccionada: Región 1**

**Justificación de la elección:**

* La **Región 1** presentó el **mayor beneficio promedio estimado** con un valor de aproximadamente **$4.54 millones de USD**, superando a las demás regiones tanto en rentabilidad como en estabilidad.
* Su **riesgo de pérdida** fue de **1.4 %**, el único que **cumple con la política empresarial** (< 2.5 %).
* El **intervalo de confianza del 95 %** para el beneficio se mantuvo **positivo**:
  entre **$0.49 M y $8.46 M**, lo que demuestra que incluso en los escenarios más conservadores, la región sigue siendo rentable.
* El modelo logró una **precisión alta (RMSE ≈ 0.89)** y un ajuste consistente entre los valores reales y predichos, lo que da confianza en las proyecciones.

---

## **Cómo el bootstrapping respaldó la decisión**

Se realizaron **1000 simulaciones de remuestreo** por región.
En cada una, se tomaron **500 pozos aleatorios** y se seleccionaron los **200 con mayor volumen predicho**.
Este proceso permitió construir una **distribución de beneficios** que reflejó la variabilidad esperada de los resultados.

Gracias a esta técnica, se estimó:

* El **beneficio promedio esperado** de la Región 1 (≈ $4.54 M).
* Un **riesgo de pérdida mínimo (1.4 %)**.
* Un **intervalo de confianza completamente positivo**, confirmando estabilidad económica.

El *bootstrapping* fue esencial para validar que la rentabilidad no se debía al azar, sino a un patrón consistente del modelo.

---



### **Conclusión general**

La **Región 1** es la **opción óptima** para el desarrollo de los 200 nuevos pozos petrolíferos.
Combina **alta rentabilidad esperada**, **riesgo mínimo**, y **resultados estables** validados mediante *bootstrapping*.
La estrategia recomendada es avanzar con la exploración de esta región.
